# Chapter 3 - Linear Regression Laboratory

In [ ]:
import pandas as pd
import numpy as np

# Modeling
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.graphics.regressionplots import *
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-white')
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [ ]:
# temp fix from https://nbviewer.jupyter.org/gist/thatneat/10286720
def transform_exog_to_model(fit, exog):
    transform=True
    self=fit

    # The following is lifted straight from statsmodels.base.model.Results.predict()
    if transform and hasattr(self.model, 'formula') and exog is not None:
        from patsy import dmatrix
        exog = dmatrix(self.model.data.orig_exog.design_info.builder,
                       exog)

    if exog is not None:
        exog = np.asarray(exog)
        if exog.ndim == 1 and (self.model.exog.ndim == 1 or
                               self.model.exog.shape[1] == 1):
            exog = exog[:, None]
        exog = np.atleast_2d(exog)  # needed in count model shape[1]

    # end lifted code
    return exog

## Simple Linear Regression

The `ISLR2` contains the `Boston`  data set, which records `medv` (median house value) for $506$ census tracts in Boston. We will seek to predict `medv` using $12$ predictors such as `rmvar` (average number of  rooms per house), `age` (average age of houses), and `lstat` (percent of households with low socioeconomic status).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Boston = pd.read_csv("/content/drive/MyDrive/Lab/Data/Boston.csv", index_col='Unnamed: 0')
Boston.index = Boston.index - 1 
Boston.head()

In [ ]:
print(Boston.shape)
print(Boston.info())

We will start by using the `ols()` function to fit a simple  linear regression model, with `medv` as the response and `lstat`  as the predictor.

 The basic syntax is $ols(y \sim x, data)$, where `y` is the response, `x` is the predictor, and `data` is the data set in which these two variables are kept.

In [ ]:
# est = smf.ols(y ~ x, data)
est = smf.ols('medv ~ lstat',data = Boston).fit()
print(est.summary())

Another way is to use scikit-learn like API as follows:

In [ ]:
X = Boston["lstat"]
X = sm.add_constant(X)
y = Boston["medv"]
model = sm.OLS(y,X).fit()
print(model.summary())

When `statsmodel` detected as a categorical variable, and thus each of its different values are treated as different entities.
An integer column can be forced to be treated as categorical using:
`
model = ols('VIQ ~ C(Gender)', data).fit()
`
By default, statsmodels treats a categorical variable with `K` possible values as `K-1` ‘dummy’ boolean variables (the last level being absorbed into the intercept term). This is almost always a good default choice - however, it is possible to specify different encodings for categorical variables (http://statsmodels.sourceforge.net/devel/contrasts.html).

In order to obtain a confidence interval for the coefficient estimates, we can use the `conf_int()` command.

In [ ]:
est.conf_int(alpha=0.05)      # default alpha=0.05 : 95% confidence interval

The `predict()` function can be used to produce the prediction for new instance.

In [ ]:
X_new = pd.DataFrame({'lstat':[5,10,15]})
est.predict(X_new)

In [ ]:
# prediction interval: _, lower bound, upper bound
transformed = transform_exog_to_model(est, X_new)
wls_prediction_std(est, transformed , weights=[1])[1:]

The `get_prediction()` function can be used to produce confidence intervals and prediction intervals for the prediction of `medv` for a given value of `lstat`.

In [ ]:
predictions = est.get_prediction(X_new)
predictions.summary_frame(alpha=0.05)

For instance, the 95\,\% confidence interval associated with a `lstat` value of 10 is $(24.47, 25.63)$, and the 95\,\% prediction interval is $(12.828, 37.28)$.
As expected, the confidence and prediction intervals are centered around the same point (a predicted value of $25.05$ for `medv` when `lstat` equals 10), but the latter are substantially wider.

We will now plot `medv` and `lstat` along with the least squares regression line using `matplotlib` or `regplot()` functions.

In [ ]:
sns.scatterplot(x='lstat', y='medv', data=Boston)

X = pd.DataFrame({'lstat':[Boston.lstat.min(), Boston.lstat.max()]})
Y_pred = est.predict(X)
sns.lineplot(x=X.values[:,0], y=Y_pred.values, color='red')
plt.xlabel("lstat")
plt.ylabel("medv")

In [ ]:
sns.regplot(x='lstat',y='medv', data=Boston)

Next we examine some diagnostic plots, several of which were discussed in Section 3.3.3. Four diagnostic plots are plotted according to the results from `ols()`. Also check https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.OLSInfluence.html and https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.html#statsmodels.regression.linear_model.OLSResults

In [ ]:
infulence = OLSInfluence(est)

In [ ]:
ols_sm_resid = est.resid # residuals
ols_fitted = est.fittedvalues
prstd = wls_prediction_std(est)[0]
ols_sm_resid_stud = ols_sm_resid / prstd # studentized residuals or infulence.resid_studentized_internal

In [ ]:
infulence = OLSInfluence(est)
ols_sm_resid = est.resid # residuals
ols_fitted = est.fittedvalues
ols_sm_resid_stud = infulence.resid_studentized_internal
leverage = OLSInfluence(est).hat_matrix_diag


f, axes = plt.subplots(2, 2, sharex=False, sharey=False) 
f.set_figheight(15)
f.set_figwidth(20)

sns.regplot(x='lstat', y='medv', data=Boston, ax=axes[0, 0], scatter_kws={'alpha': 0.5}) # regression plot
axes[0, 0].set_title("reg plot")
sns.scatterplot(x=ols_fitted,y=ols_sm_resid, ax=axes[0, 1], alpha=0.5)
axes[0, 1].set_xlabel("fittedvalues")
axes[0, 1].set_ylabel("residual")
axes[0, 1].set_title("residual plot")
#sns.residplot(x=est.predict(), y='medv', data=df, ax=axes[0, 1], scatter_kws={'alpha': '0.5'}) # residual plot

#plot_leverage_resid2(ols_sm_results, ax=axes[1, 0], color='red') # leverage plot

# custom leverage plot instead of above
#axes[1, 0].autoscale(enable=True, axis='y', tight=True)
axes[1, 0].scatter(leverage, ols_sm_resid_stud, alpha=0.5, color='red')
axes[1, 0].set_xlabel("Leverage")
axes[1, 0].set_ylabel("Studentized residuals")
#axes[1, 0].set_ylim(-5, 5)
axes[1, 0].set_title("leverage")
# studentized residual plot
axes[1, 1].scatter(ols_fitted, ols_sm_resid_stud, alpha=0.5, color='magenta')
axes[1, 1].axhline(0, ls=":", c=".2")
axes[1, 1].axhline(-3, ls=":", c=".6")
axes[1, 1].axhline(3, ls=":", c=".6")
axes[1, 1].set_ylim(-5, 5)
axes[1, 1].set_xlabel("fittedvalues")
axes[1, 1].set_ylabel("Studentized residuals")
axes[1, 1].set_title("studentized residual plot")

x = est.fittedvalues[np.logical_or(ols_sm_resid_stud > 3, ols_sm_resid_stud < -3)]
y = ols_sm_resid_stud[np.logical_or(ols_sm_resid_stud > 3, ols_sm_resid_stud < -3)]

for i, x, y in zip(x.index, x, y):
    axes[1, 1].annotate(i, xy=(x, y));

### Optional - Other useful plot
Seaborn also has the functionality of residual plot


In [ ]:
sns.residplot(x="lstat", y="medv", data=Boston)



Statsmodel has more diagonostic plot, like the influence plot where the size of the points is relate to Cook's distance. https://www.statsmodels.org/stable/examples/notebooks/generated/regression_plots.html

In [ ]:
f = sm.graphics.influence_plot(est, criterion="cooks")
f.set_figheight(10)
f.set_figwidth(10)

The `plot_regress_exog` function is a convenience function that gives a 2x2 plot containing the dependent variable and fitted values with confidence intervals vs. the independent variable chosen, the residuals of the model vs. the chosen independent variable, a partial regression plot, and a CCPR plot. This function can be used for quickly checking modeling assumptions with respect to a single regressor. Check https://www.statsmodels.org/stable/examples/notebooks/generated/regression_plots.html#Component-Component-plus-Residual-(CCPR)-Plots

In [ ]:
f = sm.graphics.plot_regress_exog(est, "lstat")
f.set_figheight(10)
f.set_figwidth(15)
f.tight_layout(pad=1.0)

## Multiple Regression

In order to fit a multiple linear regression model using least squares, we again use the `ols()` function. The syntax $ols(y \sim x1 + x2 + x3)$ is used to fit a model with three predictors, `x1`, `x2`, and `x3`. The `summary()` function now outputs the regression coefficients for all the predictors.

In [ ]:
#string_cols = ' + '.join(data.columns[:-1])
est = smf.ols('medv ~ lstat+age',data = Boston).fit()
print(est.summary())

The `Boston` data set contains 12 variables, and so it would be cumbersome to have to type all of these in order to perform a regression using all of the predictors.
Instead, we can use the code:

In [ ]:
columns_selected = "+".join(Boston.columns.difference(["medv"]))
my_formula = "medv ~ " + columns_selected
est = smf.ols(my_formula,data = Boston).fit()
print(est.summary())

We can access the individual components of a summary object by name. Hence `est.rsquared` gives us the $R^2$.
The `vif()` function can be used to compute variance inflation factors.  Most VIF's are low to moderate for this data. Check https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html

In [ ]:
est.rsquared

In [ ]:
# don't forget to add constant if the ols model includes intercept
boston = Boston.drop('medv', axis=1).assign(const=1)
boston.head()

In [ ]:
# variance inflation factors

for i, col in enumerate(boston.columns):
    if col == 'const':
        pass
    elif len(col) > 6:
        print(col, ':', "{0:.2f}".format(vif(boston.to_numpy(), i)))
    else:
        print(col, '\t:', "{0:.2f}".format(vif(boston.to_numpy(), i)))

What if we would like to perform a regression using all of the variables but one?  For example, in the above regression output,  `age` has a high $p$-value. So we may wish to run a regression excluding this predictor. The following procedure results in a regression using all predictors except `age`.

In [ ]:
columns_selected = "+".join(Boston.columns.difference(["medv", "age"]))
my_formula = "medv ~ " + columns_selected
lm_fit1 = smf.ols(formula = my_formula, data=Boston).fit()
lm_fit1.summary().tables[1]

## Interaction term

It is easy to include interaction terms in a linear model using the `ols()` function. The syntax `lstat:age` tells `Python` to include an interaction term between `lstat` and `age`. The syntax `lstat * age` simultaneously includes `lstat`, `age`, and the interaction term `lstat`$\times$`age` as predictors; it is a shorthand for `lstat + age + lstat:age`.

In [ ]:
est = smf.ols('medv ~ lstat*age',data = Boston).fit()
print(est.summary())

## Non-linear Transformations of the Predictors

The `ols()` function can also accommodate non-linear transformations of the predictors. For instance, given a predictor $X$, we can create a predictor $X^2$ using  `I(X**2)`. The function `I()` is needed since the `**` has a special meaning in a formula object. We now perform a regression of `medv` onto `lstat` and `lstat^2`.

In [ ]:
#adding power term
est = smf.ols('medv ~ lstat + I(lstat**2)',data = Boston).fit()
print(est.summary())

The near-zero $p$-value associated with the quadratic term suggests that it leads to an improved model. We use the `anova()` function to further quantify the extent to which the quadratic fit is superior to the linear fit.

In [ ]:
est2 = smf.ols('medv ~ lstat', data = Boston).fit()
sm.stats.anova_lm(est2, est, typ=1)

Here Model 0 represents the linear submodel containing only one predictor, `lstat`, while Model 1 corresponds to the larger quadratic model that has two predictors, `lstat` and `lstat^2`. The `anova()` function performs a hypothesis test
comparing the two models. The  null hypothesis is that the two models fit the data equally well,  and the alternative hypothesis is that the full model is superior. Here the $F$-statistic is $135$  and the associated $p$-value is virtually zero. This provides very clear evidence that the model containing the predictors `lstat` and `lstat^2` is far superior to the model that only contains the predictor `lstat`. This is not surprising, since earlier we saw evidence for non-linearity in the relationship between `medv` and `lstat`. If we type

In order to create a cubic fit, we can include a predictor of the form `I(X**3)`. However, this approach can start to get cumbersome for higher-order polynomials. A better approach involves using the `PolynomialFeatures()` function to create the polynomial within `ols()`. For example, the following command produces a fifth-order polynomial fit:

In [ ]:
polynomial_features= PolynomialFeatures(degree=5) # using sklearn
xp = polynomial_features.fit_transform(Boston.lstat.values.reshape(-1,1))[:,1:] #the intercept should be removed first

In [ ]:
ols_smf = smf.ols(formula='medv ~ xp', data=Boston)
ols_smf_results = ols_smf.fit()
print(ols_smf_results.summary())

This suggests that including additional  polynomial terms, up to fifth order, leads to an improvement in the model fit! However, further investigation of the data reveals that no polynomial terms beyond fifth order have significant $p$-values
in a regression fit.

Of course, we are in no way restricted to using polynomial transformations of the predictors. Here we try a log transformation.

In [ ]:
# polynomial ols model with intercept
ols_smf = smf.ols(formula='medv ~ np.log(rm)', data=Boston)

# fitted model and summary
ols_smf_results = ols_smf.fit()
print(ols_smf_results.summary())

## Qualitative predictors

We will now examine the `Carseats` data, which is part of the `ISLR2`. We will  attempt to predict `Sales`(child car seat sales) in $400$ locations based on a number of predictors.

In [ ]:
Carseats = pd.read_csv("/content/drive/MyDrive/Lab/Data/Carseats.csv")
print(Carseats.shape)
Carseats.head()

The `Carseats` data includes qualitative predictors such as `shelveloc`, an indicator of the quality of the shelving location---that is, the  space within a store in which the car seat is displayed---at each location. The predictor `shelveloc` takes on three possible values:  *Bad*, *Medium*, and *Good*. Given a qualitative variable such as `shelveloc`, `Python` generates dummy variables automatically. Below we fit a multiple regression model that includes some interaction terms. The syntax `X1:XP` tells `Python` to include an interaction term between `X1` and `XP`.

In [ ]:
# ols model with intercept
columns_selected = "+".join(Carseats.columns.difference(["Sales"]))
my_formula = "Sales ~ Income:Advertising + Price:Age + " + columns_selected  

# fitted model and summary
lm_fit = smf.ols(my_formula, data=Carseats).fit()
print(lm_fit.summary())

`Python` has created a `ShelveLoc[T.Good]` dummy variable that takes on a value of 1 if the shelving location is good, and 0 otherwise. It has also created a `ShelveLoc[T.Medium]` dummy variable that equals 1 if the shelving location is medium, and 0 otherwise. A bad shelving location corresponds to a zero for each of the two dummy variables.
The fact that the coefficient for `ShelveLoc[T.Good]` in the regression output is positive indicates that a good shelving location is associated with high sales (relative to a bad location). And `ShelveLoc[T.Medium]` has a smaller positive coefficient, indicating that a medium shelving location is associated with higher sales than a bad shelving location but lower sales than a good shelving location.

Also check `pd.get_dummies` (https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

## The sklearn is another popular way for performing OLS in Python

Check `sklearn` (https://scikit-learn.org/stable/modules/linear_model.html)

In [ ]:
# ols model with intercept
ols_sl = linear_model.LinearRegression(fit_intercept=True) 

# fitted ols model (.values.reshape(-1, 1) is required for single predictor?)
x_train = Boston.lstat.values.reshape(-1, 1)
y_true =  Boston.medv
ols_sl.fit(x_train, y_true)

y_pred = ols_sl.predict(x_train)
# summary
ols_sl.intercept_, ols_sl.coef_

In [ ]:
residual = y_true - y_pred

In [ ]:
ax = Boston.plot.scatter(x='lstat', y='medv', figsize=(8, 8))
ax.plot(Boston.lstat, y_pred)
for x, yactual, yfitted in zip(Boston.lstat, Boston.medv, y_pred): 
    ax.plot((x, x), (yactual, yfitted), '--', color='C1')

plt.tight_layout()
plt.show()

In [ ]:
ols_sl_summary = {'R2': r2_score(y_true, y_pred), 
                  'Ex. Var': explained_variance_score(y_true, y_pred), 
                  'MSE': mean_squared_error(y_true, y_pred)}

for k, v in ols_sl_summary.items():
    print(k, ':', v)

In [ ]:
# out-of-sample predictions
ols_sl.predict(np.array([5, 10, 15]).reshape(-1, 1))


### Optional - Visualizer for sklearn
Sklearn do not come with statistical visulizer like seaborn but you can use yellowbrick (https://www.scikit-yb.org/en/latest/)

In [ ]:
!pip install -U yellowbrick #besure to upgrade your yellowbrick to above 1.0

In [ ]:
from yellowbrick.regressor import PredictionError, ResidualsPlot, CooksDistance

In [ ]:
model = linear_model.LinearRegression(fit_intercept=True)
visualizer = PredictionError(model)
visualizer.fit(x_train, y_true)  # Fit the training data to the visualizer
visualizer.score(x_train, y_true)
visualizer.show()

In [ ]:
visualizer = ResidualsPlot(model, is_fitted=True)
visualizer.score(x_train, y_true)  # Evaluate the model on the test data
visualizer.show()                 # Finalize and render the figure

Histogram can be replaced with a Q-Q plot, which is a common way to check that residuals are normally distributed. If the residuals are normally distributed, then their quantiles when plotted against quantiles of normal distribution should form a straight line. 

In [ ]:
plt.figure(figsize=(12,8)) 
visualizer = ResidualsPlot(model, hist=False, qqplot=True, is_fitted=True)
visualizer.score(x_train, y_true)  # Evaluate the model on the test data
visualizer.show() 

In [ ]:
visualizer = CooksDistance()
visualizer.fit(x_train, y_true)
visualizer.show()